In [ ]:
1

In [ ]:
import numpy as np
import pandas as pd

pd.options.display.float_format = (
    None  # lambda v: "{:_.0f}".format(v).replace("_", " ")
)

In [ ]:
from openfisca_france import CountryTaxBenefitSystem

In [ ]:
from sports import compute, fields, get_df, get_results, build_data

In [ ]:
import sys

sys.path.append("../technique")
from utils import *

In [ ]:
df = get_df()

In [ ]:
df.shape

In [ ]:
base = CountryTaxBenefitSystem()
base.load_extension("openfisca_france_local")

In [ ]:
from scenario import QfFiscalReform, CEReform, period

qf_fiscal_reform = QfFiscalReform(base)
ce_reform = CEReform(base)
reform = ce_reform

In [ ]:
r = get_results(base, 5, reform)
r

In [ ]:
done_indexes = {}
for v in fields:
    done_indexes[v] = ~df[v].isna()

In [ ]:
done = pd.DataFrame(data=done_indexes)

In [ ]:
df[(~done.any(axis=1)) * df.exclu.isna()]  # *(df.type == "Piscine")]

In [ ]:
for v in fields:
    print(v)
    data, res = build_data(df, v, 1)
    openfisca_output_variable = fields[v]["openfisca_output_variable"]
    compute(base, data, res, openfisca_output_variable)
    display(res[~res.res])
    assert res[~res.res].shape[0] == 0

In [ ]:
res

In [ ]:
def build_sample(product_df, sample_count):
    count = sum(product_df.quantité)
    sample_ids = np.repeat(list(range(sample_count)), count)
    sample_qfrule = np.tile(
        np.repeat(product_df.qfrule, product_df.quantité), sample_count
    )
    sample_individu_df = pd.DataFrame(
        {
            "sample_id": sample_ids,
            "famille_id": list(range(count * sample_count)),
            "agerule": np.tile(
                np.repeat(product_df.age, product_df.quantité), sample_count
            ),
            "taux_incapacite": np.tile(
                np.repeat(
                    np.where(product_df.qfrule.str.contains("HANDICAP"), 0.8, 0),
                    product_df.quantité,
                ),
                sample_count,
            ),
        }
    )

    determine_age(sample_individu_df)

    sample_famille_df = pd.DataFrame(
        {
            "qfrule": sample_qfrule,
        }
    )
    determine_qf(sample_famille_df)

    sample_menage_df = pd.DataFrame({})
    sample_foyerfiscaux_df = pd.DataFrame({})

    sample_individu_df["famille_role_index"] = 0
    sample_individu_df["foyer_fiscal_id"] = sample_individu_df.famille_id
    sample_individu_df["foyer_fiscal_role_index"] = 0
    sample_individu_df["menage_id"] = sample_individu_df.famille_id
    sample_individu_df["menage_role_index"] = 0

    sample_data = dict(
        input_data_frame_by_entity=dict(
            individu=sample_individu_df,
            famille=sample_famille_df,
            menage=sample_menage_df,
            foyer_fiscal=sample_foyerfiscaux_df,
        )
    )

    return sample_data, sample_ids


def compute_sample(categorie, sample_count=2):
    champ_pu = fields[categorie]["champ_pu"]
    openfisca_input_variable = fields[categorie]["openfisca_input_variable"]
    openfisca_output_variable = fields[categorie]["openfisca_output_variable"]

    product_df = df[~df[categorie].isna()]

    sample_data, sample_ids = build_sample(product_df, sample_count)
    prix = np.tile(np.repeat(product_df[champ_pu], product_df.quantité), sample_count)

    sample_data["input_data_frame_by_entity"]["individu"][openfisca_input_variable] = 1

    sample_scenario = StrasbourgSurveyScenario(base, data=sample_data)
    sample_prix = sample_scenario.simulation.calculate(
        openfisca_output_variable, base_period
    )

    sample_res_df = pd.DataFrame(
        data={
            "prestation": np.tile(
                np.repeat(product_df.prestation, product_df.quantité), sample_count
            ),
            "categorie": np.tile(
                np.repeat(product_df.categorie, product_df.quantité), sample_count
            ),
            "index": np.tile(
                np.repeat(product_df.index, product_df.quantité), sample_count
            ),
            "sample_id": sample_ids,
            "input_prix": prix,
            "output_prix": sample_prix,
        }
    )
    return sample_res_df

In [ ]:
categorie = "entrée unitaire"
v = 1  # compute_sample(categorie)
v

In [ ]:
df_interest = df[df.categorie.isin(fields.keys())]

In [ ]:
sample_data, sample_ids = build_sample(df[df.categorie.isin(fields.keys())], 2)

In [ ]:
sample_scenario = StrasbourgSurveyScenario(base, data=sample_data)

In [ ]:
if False:
    results = pd.DataFrame(
        data={
            v: sample_scenario.simulation.calculate(
                fields[v]["openfisca_output_variable"], base_period
            )
            for v in fields
        }
    )
    results

In [ ]:
# results[df_interest.categorie]

In [ ]:
# v.groupby(by=["sample_id", "categorie"]).sum(numeric_only=True)

In [ ]:
df[df.categorie == "patinoire entree unitaire"]["euros"].sum()